<a href="https://colab.research.google.com/github/hannahgz/BenchmarkRCStrategies/blob/master/FullBinaryExampleNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
#Get hg19 fasta by download 2bit and then converting to fa
![[ -f hg19.2bit ]] || wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit -O hg19.2bit  
![[ -f twoBitToFa ]] || wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa -O twoBitToFa
!chmod a+x twoBitToFa
![[ -f hg19.genome.fa ]] || ./twoBitToFa hg19.2bit hg19.genome.fa

--2021-01-26 00:59:11--  http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 816241703 (778M)
Saving to: ‘hg19.2bit’

hg19.2bit           100%[===================>] 778.43M  20.9MB/s    in 39s     

2021-01-26 00:59:50 (19.9 MB/s) - ‘hg19.2bit’ saved [816241703/816241703]



In [50]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 577 kB of archives.
After this operation, 2,040 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 bedtools amd64 2.26.0+dfsg-5 [577 kB]
Fetched 577 kB in 2s (300 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../bedtools_2.26.0+dfsg-5_amd64.deb ...
Unpacking bedtools (2.26.0+dfsg-5) ...
Setting up bedtools (2.26.0+dfsg-5) ...


In [51]:
import gzip
![[ -f SPI1_foreground.bed.gz ]] || wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/for_henry/tf_data/Spi1/foreground.bed.gz -O SPI1_foreground.bed.gz

#only want the first 3 columns of the bed file which include chromosome and peak location
![[ -f SPI1_foreground_actual.bed.gz ]] || zcat SPI1_foreground.bed.gz | perl -lane 'print $F[0]."\t".$F[1]."\t".$F[2]' | sortBed | gzip -c > SPI1_foreground_actual.bed.gz
#training set consists of all regions except those on chr1 and chr2
![[ -f SPI1_foreground_train.bed.gz ]] || zcat SPI1_foreground_actual.bed.gz | egrep -w -v 'chr1|chr2' | gzip -c > SPI1_foreground_train.bed.gz
#validation set consists of all regions on chr1
![[ -f SPI1_foreground_val.bed.gz ]] || zcat SPI1_foreground_actual.bed.gz | egrep -w 'chr1' | gzip -c > SPI1_foreground_val.bed.gz
#test set consists of all regions on chr2
![[ -f SPI1_foreground_test.bed.gz ]] || zcat SPI1_foreground_actual.bed.gz | egrep -w 'chr2' | gzip -c > SPI1_foreground_test.bed.gz

--2021-01-26 00:48:48--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/for_henry/tf_data/Spi1/foreground.bed.gz
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 867245 (847K) [application/x-gzip]
Saving to: ‘SPI1_foreground.bed.gz’

SPI1_foreground.bed 100%[===================>] 846.92K  1.21MB/s    in 0.7s    

2021-01-26 00:48:49 (1.21 MB/s) - ‘SPI1_foreground.bed.gz’ saved [867245/867245]



In [52]:
#do the same for the background file
![[ -f SPI1_background.bed.gz ]] || wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/for_henry/tf_data/Spi1/background.bed.gz -O SPI1_background.bed.gz

![[ -f SPI1_background_actual.bed.gz ]] || zcat SPI1_background.bed.gz | perl -lane 'print $F[0]."\t".$F[1]."\t".$F[2]' | sortBed | gzip -c > SPI1_background_actual.bed.gz
#train set
![[ -f SPI1_background_train.bed.gz ]] || zcat SPI1_background_actual.bed.gz | egrep -w -v 'chr1|chr2' | gzip -c > SPI1_background_train.bed.gz
#validation set
![[ -f SPI1_background_val.bed.gz ]] || zcat SPI1_background_actual.bed.gz | egrep -w 'chr1' | gzip -c > SPI1_background_val.bed.gz
#test set
![[ -f SPI1_background_test.bed.gz ]] || zcat SPI1_background_actual.bed.gz | egrep -w 'chr2' | gzip -c > SPI1_background_test.bed.gz

--2021-01-26 00:48:55--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/for_henry/tf_data/Spi1/background.bed.gz
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3784041 (3.6M) [application/x-gzip]
Saving to: ‘SPI1_background.bed.gz’

SPI1_background.bed 100%[===================>]   3.61M  3.34MB/s    in 1.1s    

2021-01-26 00:48:56 (3.34 MB/s) - ‘SPI1_background.bed.gz’ saved [3784041/3784041]



In [53]:
!zcat SPI1_foreground.bed.gz | head

chr17	66493802	66494802	idrChipSeqPeaks	chr17:66494123-66494471
chr11	117881374	117882374	idrChipSeqPeaks	chr11:117881603-117882133
chr1	192577938	192578938	idrChipSeqPeaks	chr1:192578265-192578639
chr12	24858378	24859378	idrChipSeqPeaks	chr12:24858665-24859058
chr19	35695893	35696893	idrChipSeqPeaks	chr19:35696206-35696655
chr13	53495039	53496039	idrChipSeqPeaks	chr13:53495355-53495776
chr2	60666201	60667201	idrChipSeqPeaks	chr2:60666502-60666880
chr18	3177598	3178598	idrChipSeqPeaks	chr18:3177902-3178305
chr14	68734604	68735604	idrChipSeqPeaks	chr14:68734937-68735268
chr9	126964391	126965391	idrChipSeqPeaks	chr9:126964700-126965152


In [55]:
!zcat SPI1_foreground_val.bed.gz | head

chr1	839625	840625
chr1	841786	842786
chr1	937248	938248
chr1	961668	962668
chr1	970836	971836
chr1	973753	974753
chr1	999369	1000369
chr1	1079129	1080129
chr1	1147949	1148949
chr1	1166399	1167399


In [56]:
#create lookup file, concatenating 1 to the end of foreground file for the pos bed file
#concatenate 0 to the end of the background file for the neg bed file
![[ -f SPI1_foreground_lookup.bed.gz ]] || zcat SPI1_foreground.bed.gz | perl -lane 'print $F[0]."\t".$F[1]."\t".$F[2]."\t".1' |  gzip -c > SPI1_foreground_lookup.bed.gz
![[ -f SPI1_background_lookup.bed.gz ]] || zcat SPI1_background.bed.gz | perl -lane 'print $F[0]."\t".$F[1]."\t".$F[2]."\t".0' | gzip -c > SPI1_background_lookup.bed.gz
!cat SPI1_foreground_lookup.bed.gz SPI1_background_lookup.bed.gz > SPI1_lookup.bed.gz

In [57]:
!zcat SPI1_lookup.bed.gz | head

chr17	66493802	66494802	1
chr11	117881374	117882374	1
chr1	192577938	192578938	1
chr12	24858378	24859378	1
chr19	35695893	35696893	1
chr13	53495039	53496039	1
chr2	60666201	60667201	1
chr18	3177598	3178598	1
chr14	68734604	68735604	1
chr9	126964391	126965391	1


In [103]:
!wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Spi1/valid_data.hdf5
!wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Spi1/test_data.hdf5

--2021-01-26 03:35:00--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Spi1/valid_data.hdf5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16421649 (16M)
Saving to: ‘valid_data.hdf5’

valid_data.hdf5     100%[===================>]  15.66M  9.56MB/s    in 1.6s    

2021-01-26 03:35:02 (9.56 MB/s) - ‘valid_data.hdf5’ saved [16421649/16421649]

--2021-01-26 03:35:02--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Spi1/test_data.hdf5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13756995 (13M)
Saving to: ‘test_data.hdf5’

test_data.hdf5      100%[===================>]  13.12M  8.80MB/s    in 1.5s    

2

In [59]:
!pip install momma_dragonn

In [104]:
import momma_dragonn

valid_data_loader = momma_dragonn.data_loaders.hdf5_data_loader.MultimodalAtOnceDataLoader(
    path_to_hdf5="valid_data.hdf5", strip_enclosing_dictionary=True)
valid_data = valid_data_loader.get_data()
    
test_data_loader = momma_dragonn.data_loaders.hdf5_data_loader.MultimodalAtOnceDataLoader(
    path_to_hdf5="test_data.hdf5", strip_enclosing_dictionary=True)
test_data = test_data_loader.get_data()

/usr/local/lib/python3.6/dist-packages/momma_dragonn/data_loaders/hdf5_data_loader.py:32: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.f = h5py.File(self.path_to_hdf5)


Input modes ['sequence']
Output modes ['output']
Input modes ['sequence']
Output modes ['output']


In [90]:
Coordinates = namedtuple("Coordinates",
                         ["chrom", "start", "end", "isplusstrand"])
Coordinates.__new__.__defaults__ = (True,)


def apply_mask(tomask, mask):
    if isinstance(tomask, dict):
        return dict([(key, val[mask]) for key,val in tomask.items()])
    elif isinstance(tomask, list):
        return [x[mask] for x in mask]
    else:
        return x[mask]


class KerasBatchGenerator(keras.utils.Sequence):
  
    """
    Args:
        coordsbatch_producer (KerasSequenceApiCoordsBatchProducer)
        inputs_coordstovals (CoordsToVals)
        targets_coordstovals (CoordsToVals)
        sampleweights_coordstovals (CoordsToVals)
        coordsbatch_transformer (AbstracCoordBatchTransformer)
        qc_func (callable): function that can be used to filter
            out poor-quality sequences.
        sampleweights_coordstoval: either this argument or
            sampleweights_from_inputstargets could be used to
            specify sample weights. sampleweights_coordstoval
            takes a batch of coords as inputs.
        sampleweights_from_inputstargets: either this argument or
            sampleweights_coordstoval could be used to
            specify sample weights. sampleweights_from_inputstargets
            takes the inputs and targets values to generate the weights.
    """
    def __init__(self, coordsbatch_producer,
                       inputs_coordstovals,
                       targets_coordstovals,
                       coordsbatch_transformer=None,
                       qc_func=None,
                       sampleweights_coordstovals=None,
                       sampleweights_from_inputstargets=None):
        self.coordsbatch_producer = coordsbatch_producer
        self.inputs_coordstovals = inputs_coordstovals
        self.targets_coordstovals = targets_coordstovals
        self.coordsbatch_transformer = coordsbatch_transformer
        self.sampleweights_coordstovals = sampleweights_coordstovals
        self.sampleweights_from_inputstargets =\
            sampleweights_from_inputstargets
        if sampleweights_coordstovals is not None:
            assert sampleweights_from_inputstargets is None
        if sampleweights_from_inputstargets is not None:
            assert sampleweights_coordstovals is None
        self.qc_func = qc_func
 
    def __getitem__(self, index):
        coords_batch = self.coordsbatch_producer[index]
        if (self.coordsbatch_transformer is not None):
            coords_batch = self.coordsbatch_transformer(coords_batch)
        inputs = self.inputs_coordstovals(coords_batch)
        if (self.targets_coordstovals is not None):
            targets = self.targets_coordstovals(coords_batch)
        else:
            targets=None
        if (self.qc_func is not None):
            qc_mask = self.qc_func(inputs=inputs, targets=targets)
            inputs = apply_mask(tomask=inputs, mask=qc_mask)
            if (targets is not None):
                targets = apply_mask(tomask=targets, mask=qc_mask)
        else:
            qc_mask = None
        if (self.sampleweights_coordstovals is not None):
            sample_weights = self.sampleweights_coordstovals(coords_batch)
            return (inputs, targets, sample_weights)
        elif (self.sampleweights_from_inputstargets is not None):
            sample_weights = self.sampleweights_from_inputstargets(
                                inputs=inputs, targets=targets)
            return (inputs, targets, sample_weights)
        else:
            if (self.targets_coordstovals is not None):
                return (inputs, targets)
            else:
                return inputs
   
    def __len__(self):
        return len(self.coordsbatch_producer)
    
    def on_epoch_end(self):
        self.coordsbatch_producer.on_epoch_end()
        
        
def get_new_coors_around_center(coors, center_size_to_use):
    new_coors = []
    for coor in coors:
        coor_center = int(0.5*(coor.start + coor.end))
        left_flank = int(0.5*center_size_to_use)
        right_flank = center_size_to_use - left_flank
        new_start = coor_center-left_flank
        new_end = coor_center+right_flank
        new_coors.append(Coordinates(chrom=coor.chrom,
                                     start=new_start, end=new_end,
                                     isplusstrand=coor.isplusstrand))
    return new_coors


class CoordsToVals(object):
    
    def __call__(self, coors):
        """
        Args:
            coors (:obj:`list` of :obj:`Coordinates`):
        Returns:
            numpy ndarray OR list of ndarrays OR a dict of mode_name->ndarray.
              Returns a list of ndarrays if returning multiple modes.
              Alternatively, returns a dict where key is the mode name
              and the value is the ndarray for the mode.
        """
        raise NotImplementedError()


class CoordsToValsJoiner(CoordsToVals):

    def __init__(self, coordstovals_list):
        """
        Joins batches returned by other CoordsToVals objects
        Args:
            coorstovals_list (:obj:`list` of :obj:`CoordsToVals`): List of
                CoordsToVals whose values to combine
        """
        self.coordstovals_list = coordstovals_list
                
    def __call__(self, coors):
        batch_to_return = None        
        for idx,coordstovals_obj in enumerate(self.coordstovals_list):
            the_batch = coordstovals_obj(coors=coors)
            assert the_batch is not None
            if isinstance(the_batch, dict):
                assert ((batch_to_return is None) or
                        (isinstance(batch_to_return, dict))), (
                        "coordstovals object at idx"+str(idx)
                        +" returned a dict, but previous coordstovals"
                        +" objects had a return type incompatible with this")
                if (batch_to_return is None):
                    batch_to_return = {}
                for key in the_batch:
                    assert key not in batch_to_return, (
                      "coordstovals object at idx"+str(idx)
                      +" returned a dict with a key of "+key
                      +", which collides with a pre-existing key returned by"
                      +" another coordstovals object")
                batch_to_return.update(the_batch)
            else:
                assert ((batch_to_return is None) or
                        (isinstance(batch_to_return, list))), (
                        "coordstovals object at idx"+str(idx)
                        +" returned a type incompatible with dict, but previous"
                        +" coordstovals objects had a return type of dict")
                if (isinstance(the_batch, list)==False):
                    the_batch = [the_batch]
                if (batch_to_return is None):
                    batch_to_return = []
                batch_to_return.extend(the_batch)
        if (batch_to_return is None):
            batch_to_return = []
        return batch_to_return


class AbstractSingleNdarrayCoordsToVals(CoordsToVals):

    def __init__(self, mode_name=None):
        """
        Args:
            mode_name (:obj:`str`, optional): default None. If None, then
                the return of __call__ will be a numpy ndarray. Otherwise, it
                will be a dictionary with a key of mode_name and a value being
                the numpy ndarray.
        """
        self.mode_name = mode_name
    
    def _get_ndarray(self, coors):
        """
        Args:
            coors (:obj:`list` of :obj:`Coordinates):
            
        Returns:
            numpy ndarray
        """
        raise NotImplementedError()
    
    def __call__(self, coors):
        ndarray = self._get_ndarray(coors)
        if (self.mode_name is None):
            return ndarray
        else:
            return {self.mode_name: ndarray}
        
        
class SimpleLookup(AbstractSingleNdarrayCoordsToVals):

    def __init__(self, lookup_file,
                       transformation=None,
                       default_returnval=0.0, **kwargs):
        super(SimpleLookup, self).__init__(**kwargs)
        self.lookup_file = lookup_file
        self.transformation = transformation
        self.default_returnval = default_returnval
        self.lookup = {}
        self.num_labels = None
        for line in (gzip.open(self.lookup_file) if ".gz"
                     in self.lookup_file else open(self.lookup_file)):
            (chrom, start_str, end_str, *labels) =\
              line.decode("utf-8").rstrip().split("\t")
            coord = Coordinates(chrom=chrom,
                                start=int(start_str),
                                end=int(end_str))
            labels = [(self.transformation(float(x))
                       if self.transformation is not None else float(x))
                      for x in labels] 
            self.lookup[(coord.chrom, coord.start, coord.end)] = labels
            if (self.num_labels is None):
                self.num_labels = len(labels)
            else:
                assert len(labels)==self.num_labels,(
                  "Unequal label lengths; "+str(len(labels), self.num_labels))
    
    def _get_ndarray(self, coors):
        to_return = []
        for coor in coors:
            if (coor.chrom, coor.start, coor.end) not in self.lookup:
                to_return.append(np.ones(self.num_labels)
                                 *self.default_returnval)
            else:
                to_return.append(
                    self.lookup[(coor.chrom, coor.start, coor.end)])
        return np.array(to_return)

def get_generators(dataset, seq_len, is_aug, curr_seed): 
    inputs_coordstovals = coordbased.coordstovals.fasta.PyfaidxCoordsToVals(
        genome_fasta_path="hg19.genome.fa",
        center_size_to_use= seq_len)
    
    targets_coordstovals = SimpleLookup(
        lookup_file = "SPI1_lookup.bed.gz",
        transformation = None, default_returnval = 0.0) 

    target_proportion_positives = 1/5
    
    if not is_aug: 
        standard_train_batch_generator = KerasBatchGenerator(
          coordsbatch_producer=coordbatchproducers.DownsampleNegativesCoordsBatchProducer(
              pos_bed_file = "SPI1_foreground_train.bed.gz",
              neg_bed_file = "SPI1_background_train.bed.gz",
              target_proportion_positives = target_proportion_positives, 
              batch_size=100,
              shuffle_before_epoch=True, 
              seed=curr_seed),
          inputs_coordstovals=inputs_coordstovals,
          targets_coordstovals=targets_coordstovals)
        return standard_train_batch_generator
    else: 
        aug_train_batch_generator = KerasBatchGenerator(
          coordsbatch_producer=coordbatchproducers.DownsampleNegativesCoordsBatchProducer(
              pos_bed_file = "SPI1_foreground_train.bed.gz",
              neg_bed_file = "SPI1_background_train.bed.gz",
              target_proportion_positives = target_proportion_positives, 
              batch_size=100,
              shuffle_before_epoch=True, 
              seed=curr_seed),
          inputs_coordstovals=inputs_coordstovals,
          targets_coordstovals=targets_coordstovals,
          coordsbatch_transformer=coordbatchtransformers.ReverseComplementAugmenter())
        return aug_train_batch_generator

In [85]:
!pip install keras-genomics
!pip install seqdataloader
%tensorflow_version 1.x

import keras
import tensorflow as tf
import numpy as np
import keras
from sklearn.metrics import roc_auc_score

from seqdataloader.batchproducers import coordbased
from seqdataloader.batchproducers.coordbased import coordstovals
from seqdataloader.batchproducers.coordbased import coordbatchproducers
from seqdataloader.batchproducers.coordbased import coordbatchtransformers
from seqdataloader.batchproducers.coordbased.coordbatchproducers import DownsampleNegativesCoordsBatchProducer
from seqdataloader.batchproducers.coordbased.coordbatchproducers import SimpleCoordsBatchProducer
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import AbstractCountAndProfileTransformer 
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import LogCountsPlusOne
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import SmoothProfiles
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import BigWigReader 
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import smooth_profiles
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import rolling_window
from seqdataloader.batchproducers.coordbased.core import Coordinates, KerasBatchGenerator, apply_mask
from seqdataloader.batchproducers.coordbased.coordbatchtransformers import AbstractCoordBatchTransformer
from seqdataloader.batchproducers.coordbased.coordbatchtransformers import get_revcomp
import numpy as np

import os

In [86]:
from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as kl
from keras.engine.base_layer import InputSpec
from keras_genomics.layers import RevCompConv1D
from keras import initializers

In [87]:
def get_reg_model(parameters):
    model = keras.models.Sequential()
    model.add(keras.layers.Convolution1D(
              input_shape=(1000,4), nb_filter=16, filter_length=15))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.core.Activation("relu"))
    model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.convolutional.MaxPooling1D(pool_length=parameters['pool_size'],
                                                      strides= parameters['strides']))         
    model.add(Flatten())
    model.add(keras.layers.core.Dense(output_dim=1, trainable=True,
                                    init="glorot_uniform"))
    model.add(keras.layers.core.Activation("sigmoid"))
    model.compile(optimizer = keras.optimizers.Adam(lr=0.001), 
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [91]:
parameters = {
    'filters': 16, 
    'input_length':1000, 
    'pool_size':40, 
    'strides': 20
}

model = get_reg_model(parameters)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=20, pool_size=40)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(trainable=True, units=1, kernel_initializer="glorot_uniform")`


In [92]:
epochs_to_train_for = 160 
standard_train_batch_generator = get_generators(dataset = "SPI1", 
                                                seq_len = 1000, 
                                                curr_seed = 1234, 
                                                is_aug = False)

Reading in positive bed file
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Got 35276  coords in positive bed file
Reading in negative bed file
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Got 182577  coords in negative bed file
The target proportion of positives of 0.2 requires the negative set to be subsampled by a factor of 2 which will result in a #neg of 91288
Using an offset of  0  before striding


In [94]:
from keras.callbacks import History

class AuRocCallback(keras.callbacks.Callback):
    def __init__(self, model, valid_X, valid_Y):
        self.model = model
        self.valid_X = valid_X
        self.valid_Y = valid_Y
        self.best_auroc_sofar = 0.0
        self.best_weights = None
        self.best_epoch_number = 0
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(self.valid_X)
        auroc = roc_auc_score(y_true=self.valid_Y, y_score=preds)
        if (auroc > self.best_auroc_sofar):
            self.best_weights = self.model.get_weights()
            self.best_epoch_number = epoch
            self.best_auroc_sofar = auroc

def train_model(model, curr_seed, train_data_loader, batch_generator,
                valid_data, epochs_to_train_for, upsampling):

    np.random.seed(curr_seed)
    tf.set_random_seed(curr_seed)
    
    if upsampling == False:
        early_stopping_callback = keras.callbacks.EarlyStopping(
                                  monitor='val_loss',
                                  patience=epochs_to_train_for,
                                  restore_best_weights=True)

        auroc_callback = AuRocCallback(model = model,
                                       valid_X=valid_data.X,
                                       valid_Y=valid_data.Y)
        history = History()
        loss_history = model.fit_generator(train_data_loader.get_batch_generator(),
                                           validation_data = (valid_data.X, valid_data.Y),
                                           epochs=epochs_to_train_for,
                                           steps_per_epoch=50,
                                           class_weight={0:1, 1: 4.75},
                                           callbacks=[auroc_callback, early_stopping_callback, history])
        return early_stopping_callback, auroc_callback, history, model
    else: 
        auroc_callback = AuRocCallback(model = model,
                                       valid_X=valid_data.X,
                                       valid_Y=valid_data.Y)
        history = History()
        loss_history = model.fit_generator(batch_generator,
                                           validation_data = (valid_data.X, valid_data.Y),
                                           epochs=epochs_to_train_for,
                                           steps_per_epoch=50,
                                           callbacks=[auroc_callback, history])
        return auroc_callback, history, model

In [95]:
auroc_callback, history, model = train_model(model = model, 
                                             curr_seed = 1234, 
                                             train_data_loader = None,
                                             batch_generator = standard_train_batch_generator,
                                             valid_data = valid_data, 
                                             epochs_to_train_for = epochs_to_train_for, 
                                             upsampling = True)

Epoch 1/160
50/50 [==============================] - 38s 765ms/step - loss: 0.7027 - accuracy: 0.6634 - val_loss: 0.6509 - val_accuracy: 0.7260
Epoch 2/160
50/50 [==============================] - 37s 743ms/step - loss: 0.6173 - accuracy: 0.7022 - val_loss: 0.6044 - val_accuracy: 0.7454
Epoch 3/160
50/50 [==============================] - 37s 733ms/step - loss: 0.6113 - accuracy: 0.7042 - val_loss: 0.5765 - val_accuracy: 0.7458
Epoch 4/160
50/50 [==============================] - 37s 731ms/step - loss: 0.5852 - accuracy: 0.7150 - val_loss: 0.5486 - val_accuracy: 0.7491
Epoch 5/160
50/50 [==============================] - 36s 727ms/step - loss: 0.5345 - accuracy: 0.7448 - val_loss: 0.5139 - val_accuracy: 0.7892
Epoch 6/160
50/50 [==============================] - 36s 726ms/step - loss: 0.4506 - accuracy: 0.7976 - val_loss: 0.4506 - val_accuracy: 0.8349
Epoch 7/160
50/50 [==============================] - 37s 732ms/step - loss: 0.3800 - accuracy: 0.8424 - val_loss: 0.3456 - val_accuracy:

In [105]:
model.set_weights(auroc_callback.best_weights)
print("Validation set AUROC with best-loss early stopping:",
      roc_auc_score(y_true=valid_data.Y, y_score=model.predict(valid_data.X)))
print("Test set AUROC with best-loss early stopping:",    
      roc_auc_score(y_true=test_data.Y, y_score=model.predict(test_data.X)))
model.set_weights(auroc_callback.best_weights) 
print("Validation AUROC at best-auroc early stopping:",
      roc_auc_score(y_true=valid_data.Y, y_score=model.predict(valid_data.X)))
print("Test set AUROC at best-auroc early stopping:",
      roc_auc_score(y_true=test_data.Y, y_score=model.predict(test_data.X)))

Validation set AUROC with best-loss early stopping: 0.9883717643873173
Test set AUROC with best-loss early stopping: 0.9892211739378929
Validation AUROC at best-auroc early stopping: 0.9883717643873173
Test set AUROC at best-auroc early stopping: 0.9892211739378929
